# Aims to check example of siren on full process

In [27]:
import json
import dask.dataframe as dd
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# '349599092' is in initial, partiel, new, evt
# '344981501' is in initial, partiel, new
# '6041099' is in initial, partiel

In [29]:
inpi_nature = 'PP' #* nature → ACTES/COMPTES/ETS/etc
siren_test = '349599092'
load_type = 'local' # 'aws'/'local'

In [43]:
source='insee'
insee_filename = 'StockEtablissement_utf8.csv'
siretisation_filename = 'insee_PP_all_matche.gz'

# define import paths
if load_type == 'local':
    ## INSEE on local machine
    insee_filepath = 'data'
    insee_csv_relative_filepath = "{}/{}".format(insee_filepath,insee_filename)
    insee = insee_csv_relative_filepath
    inpi_processed_import_full_path = 'data'
    siretisation_import_path = 'data'
    #matched data
    siretisation_fullpath =  "{}/{}".format(siretisation_import_path,siretisation_filename)
    matche = siretisation_fullpath
else :
    ## INSEE on AWS
    import boto3
    import Match_inpi_insee.aws_connectors as aws
    insee_filepath = "INSEE/Stock/ETS"
    insee_csv_relative_filepath = "{}/{}".format(insee_filepath,insee_filename)
    instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
    bucket = 'calfdata'
    # instanciate AWS connection
    AWS_connection = aws.aws_instantiate(instance_aws, bucket)
    insee = AWS_connection.url_instance_bucket(path_file = insee_csv_relative_filepath)
    inpi_processed_import_full_path = "INPI/TC_1/Stock_processed"
    
    siretisation_import_path = 'SIRETISATION/matche'
    siretisation_fullpath =  "{}/{}".format(siretisation_import_path,siretisation_filename)
    matche = AWS_connection.url_instance_bucket(path_file = siretisation_fullpath)

# INSEE : source data

In [31]:
usecols_=['siren',
                                   'siret',
                                   "numeroVoieEtablissement",
                                   "indiceRepetitionEtablissement",
                                   "typeVoieEtablissement",
                                   "libelleVoieEtablissement",
                                   "complementAdresseEtablissement",
                                   "codeCommuneEtablissement",
                                   "libelleCommuneEtablissement",
                                   "codePostalEtablissement",
                                   "codeCedexEtablissement",
                                   "libelleCedexEtablissement",
                                   "distributionSpecialeEtablissement",
                                   "libelleCommuneEtrangerEtablissement",
                                   "codePaysEtrangerEtablissement",
                                   "libellePaysEtrangerEtablissement"
                                   ]
dtype_={'siren': 'object',
                                 'siret': 'object',
                                 "numeroVoieEtablissement":'object',
                                   "indiceRepetitionEtablissement":'object',
                                   "typeVoieEtablissement":'object',
                                   "libelleVoieEtablissement":'object',
                                   "complementAdresseEtablissement":'object',
                                   "codeCommuneEtablissement":'object',
                                   "libelleCommuneEtablissement":'object',
                                   "codePostalEtablissement":'object',
                                   "codeCedexEtablissement":'object',
                                   "libelleCedexEtablissement":'object',
                                   "distributionSpecialeEtablissement":'object',
                                   "libelleCommuneEtrangerEtablissement":'object',
                                   "codePaysEtrangerEtablissement":'object',
                                   "libellePaysEtrangerEtablissement":'object'
                                 }

In [32]:
#load insee full data
data_insee_ = dd.read_csv(insee,
                          usecols=usecols_,
                          dtype=dtype_
                          )

In [33]:
data_insee_.head()

,siren,siret,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement
0,000325175,00032517500016,NaN,NaN,NaN,NaN,MANIHI COTE MONTAGNE TUAMOTU,98770,MANIHI,NaN,NaN,98727,NaN,NaN,NaN,NaN
1,000325175,00032517500024,NaN,1,NaN,PL,LEONCE DE SEYNES,84140,AVIGNON,NaN,NaN,84007,NaN,NaN,NaN,NaN
2,000325175,00032517500032,ECONOMIS,6,NaN,AV,FRANCOIS MAURIAC,84000,AVIGNON,NaN,NaN,84007,NaN,NaN,NaN,NaN
3,000325175,00032517500040,NaN,170,NaN,AV,DU COL DE L ANGE,13420,GEMENOS,NaN,NaN,13042,NaN,NaN,NaN,NaN
4,000325175,00032517500057,NaN,61,NaN,RUE,MARX DORMOY,13004,MARSEILLE 4,NaN,NaN,13204,NaN,NaN,NaN,NaN


In [34]:
# filter on example siren
data_insee_ = data_insee_.compute().loc[lambda x: x['siren'].isin([siren_test])]
data_insee_

,siren,siret,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement
105452,349599092,34959909200013,NaN,NaN,NaN,RUE,ST PIERRE ET MIQUELON,14380,TESSY-BOCAGE,NaN,NaN,50592,NaN,NaN,NaN,NaN
105453,349599092,34959909200021,NaN,10,NaN,RTE,DE PONTFARCY,50420,TESSY-BOCAGE,NaN,NaN,50592,NaN,NaN,NaN,NaN


# INPI : source data

In [35]:
year='2017'

initial_file_name = "{}_{}.{}".format('initial',inpi_nature,'gz')
new_file_name = "{}_{}_{}.{}".format(year,'NEW',inpi_nature.upper(),'gz')
evt_file_name = "{}_{}_{}.{}".format(year,'EVT',inpi_nature.upper(),'gz')
partiel_file_name = "{}_{}.{}".format('partiel',inpi_nature,'gz')
initial_filepath = "{}/{}".format(inpi_processed_import_full_path,initial_file_name)
new_filepath = "{}/{}".format(inpi_processed_import_full_path,new_file_name)
evt_filepath = "{}/{}".format(inpi_processed_import_full_path,evt_file_name)
partiel_filepath = "{}/{}".format(inpi_processed_import_full_path,partiel_file_name)

In [36]:
# get files

if load_type == 'local':
    pp_initial = initial_filepath
    pp_new = new_filepath
    pp_evt = evt_filepath
    pp_partiel = partiel_filepath
else :
    pp_initial = AWS_connection.url_instance_bucket(path_file = initial_filepath)
    pp_new = AWS_connection.url_instance_bucket(path_file = new_filepath)
    pp_evt = AWS_connection.url_instance_bucket(path_file = evt_filepath)
    pp_partiel = AWS_connection.url_instance_bucket(path_file = partiel_filepath)

In [37]:
## dtypes__
dtypes__ = {'Siren':'object',
            'Adresse_Ligne1': 'object',
       'Code_Commune': 'object',
       'Code_Postal': 'object',
       'Conjoint_Collab_Pseudo': 'object',
       'DAP_Adresse_Ligne1': 'object',
       'DAP_Code_Commune': 'object',
       'Date_Immatriculation': 'object',
       'Date_1re_Immatriculation': 'object',
       'Date_Radiation': 'object',
       'Date_Greffe': 'object',
       'Sans_Activité': 'object',
       'Auto-entrepreneur': 'object',
       'DAP_Adresse_Ligne3': 'object',
       'Pseudonyme': 'object'
           }

In [38]:
#load data
data_pp_initial= dd.read_csv(pp_initial,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )
# filter on example siren
data_pp_initial = data_pp_initial.compute().loc[lambda x: x['Siren'].isin([siren_test])] 
data_pp_initial

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,Sans_Activité,...,DAP_Ville,DAP_Code_Commune,DAP_Pays,Conjoint_Collab_Nom_Patronym,Conjoint_Collab_Nom_Usage,Conjoint_Collab_Pseudo,Conjoint_Collab_Prénoms,Conjoint_Collab_Date_Fin,Date_Greffe,Libelle_Evt
376279,5002,Coutances,2000A04130,349599092,P,2000-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BESNEVILLE,MADELAINE,NaN,Monique Sophie Françoise,NaN,2016-07-26,Création


In [39]:
data_pp_new= dd.read_csv(pp_new,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )

# filter on example siren
data_pp_new = data_pp_new.compute().loc[lambda x: x['Siren'].isin([siren_test])] 
data_pp_new

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,Sans_Activité,...,DAP_Ville,DAP_Code_Commune,DAP_Pays,Conjoint_Collab_Nom_Patronym,Conjoint_Collab_Nom_Usage,Conjoint_Collab_Pseudo,Conjoint_Collab_Prénoms,Conjoint_Collab_Date_Fin,Date_Greffe,Libelle_Evt
1728,1402,Caen,2017A00860,349599092,S,2017-12-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-15,Création


In [40]:
data_pp_evt= dd.read_csv(pp_evt,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )
# filter on example siren
data_pp_evt = data_pp_evt.compute().loc[lambda x: x['Siren'].isin([siren_test])] 
data_pp_evt

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,Sans_Activité,...,DAP_Ville,DAP_Code_Commune,DAP_Pays,Conjoint_Collab_Nom_Patronym,Conjoint_Collab_Nom_Usage,Conjoint_Collab_Pseudo,Conjoint_Collab_Prénoms,Conjoint_Collab_Date_Fin,Date_Greffe,Libelle_Evt
5463,5002,Coutances,2000A04130,349599092,P,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-18,Modifications relatives au dossier
5464,5002,Coutances,2000A04130,349599092,P,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BESNEVILLE,MADELAINE,NaN,Monique Sophie Françoise,NaN,2017-12-18,Modifications relatives au dossier


In [41]:
data_pp_partiel= dd.read_csv(pp_partiel,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )
# filter on example siren
data_pp_partiel = data_pp_partiel.compute().loc[lambda x: x['Siren'].isin([siren_test])] 
data_pp_partiel

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,Sans_Activité,...,DAP_Ville,DAP_Code_Commune,DAP_Pays,Conjoint_Collab_Nom_Patronym,Conjoint_Collab_Nom_Usage,Conjoint_Collab_Pseudo,Conjoint_Collab_Prénoms,Conjoint_Collab_Date_Fin,Date_Greffe,Libelle_Evt
5865,5002,Coutances,2000A04130,349599092,P,2000-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BESNEVILLE,MADELAINE,NaN,Monique Sophie Françoise,2018-05-22,2018-01-18,Création


# SIRETISATION : matched data

In [45]:
#load matched data
data_matche= dd.read_csv(matche,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )
# filter on example siren
data_matche = data_matche.compute().loc[lambda x: x['siren'].isin([siren_test])] 
data_matche

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url,source
288000,349599092,34959909200013,O,2000-01-01,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2016-07-26,both,https://data.inpi.fr/entreprises/349599092,initial
288001,349599092,34959909200021,O,2000-01-01,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2016-07-26,both,https://data.inpi.fr/entreprises/349599092,initial
288002,349599092,34959909200013,O,2017-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-15,both,https://data.inpi.fr/entreprises/349599092,new
288003,349599092,34959909200021,O,2017-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-15,both,https://data.inpi.fr/entreprises/349599092,new
288004,349599092,34959909200013,O,NaN,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/349599092,evt
288005,349599092,34959909200021,O,NaN,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/349599092,evt
288006,349599092,34959909200013,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/349599092,evt
288007,349599092,34959909200021,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/349599092,evt
288008,349599092,34959909200013,O,2000-01-01,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2018-01-18,both,https://data.inpi.fr/entreprises/349599092,partiel
288009,349599092,34959909200021,O,2000-01-01,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2018-01-18,both,https://data.inpi.fr/entreprises/349599092,partiel


# Recap of all data

In [46]:
siren_test

'349599092'

In [53]:
data_insee_

,siren,siret,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement
105452,349599092,34959909200013,NaN,NaN,NaN,RUE,ST PIERRE ET MIQUELON,14380,TESSY-BOCAGE,NaN,NaN,50592,NaN,NaN,NaN,NaN
105453,349599092,34959909200021,NaN,10,NaN,RTE,DE PONTFARCY,50420,TESSY-BOCAGE,NaN,NaN,50592,NaN,NaN,NaN,NaN


In [48]:
data_pp_initial

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,Sans_Activité,...,DAP_Ville,DAP_Code_Commune,DAP_Pays,Conjoint_Collab_Nom_Patronym,Conjoint_Collab_Nom_Usage,Conjoint_Collab_Pseudo,Conjoint_Collab_Prénoms,Conjoint_Collab_Date_Fin,Date_Greffe,Libelle_Evt
376279,5002,Coutances,2000A04130,349599092,P,2000-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BESNEVILLE,MADELAINE,NaN,Monique Sophie Françoise,NaN,2016-07-26,Création


In [49]:
data_pp_new

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,Sans_Activité,...,DAP_Ville,DAP_Code_Commune,DAP_Pays,Conjoint_Collab_Nom_Patronym,Conjoint_Collab_Nom_Usage,Conjoint_Collab_Pseudo,Conjoint_Collab_Prénoms,Conjoint_Collab_Date_Fin,Date_Greffe,Libelle_Evt
1728,1402,Caen,2017A00860,349599092,S,2017-12-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-15,Création


In [50]:
data_pp_evt

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,Sans_Activité,...,DAP_Ville,DAP_Code_Commune,DAP_Pays,Conjoint_Collab_Nom_Patronym,Conjoint_Collab_Nom_Usage,Conjoint_Collab_Pseudo,Conjoint_Collab_Prénoms,Conjoint_Collab_Date_Fin,Date_Greffe,Libelle_Evt
5463,5002,Coutances,2000A04130,349599092,P,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-18,Modifications relatives au dossier
5464,5002,Coutances,2000A04130,349599092,P,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BESNEVILLE,MADELAINE,NaN,Monique Sophie Françoise,NaN,2017-12-18,Modifications relatives au dossier


In [51]:
data_pp_partiel

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,Sans_Activité,...,DAP_Ville,DAP_Code_Commune,DAP_Pays,Conjoint_Collab_Nom_Patronym,Conjoint_Collab_Nom_Usage,Conjoint_Collab_Pseudo,Conjoint_Collab_Prénoms,Conjoint_Collab_Date_Fin,Date_Greffe,Libelle_Evt
5865,5002,Coutances,2000A04130,349599092,P,2000-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BESNEVILLE,MADELAINE,NaN,Monique Sophie Françoise,2018-05-22,2018-01-18,Création


In [52]:
data_matche

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url,source
288000,349599092,34959909200013,O,2000-01-01,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2016-07-26,both,https://data.inpi.fr/entreprises/349599092,initial
288001,349599092,34959909200021,O,2000-01-01,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2016-07-26,both,https://data.inpi.fr/entreprises/349599092,initial
288002,349599092,34959909200013,O,2017-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-15,both,https://data.inpi.fr/entreprises/349599092,new
288003,349599092,34959909200021,O,2017-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-15,both,https://data.inpi.fr/entreprises/349599092,new
288004,349599092,34959909200013,O,NaN,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/349599092,evt
288005,349599092,34959909200021,O,NaN,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/349599092,evt
288006,349599092,34959909200013,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/349599092,evt
288007,349599092,34959909200021,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/349599092,evt
288008,349599092,34959909200013,O,2000-01-01,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2018-01-18,both,https://data.inpi.fr/entreprises/349599092,partiel
288009,349599092,34959909200021,O,2000-01-01,NaN,NaN,NaN,NaN,50420,50592,NaN,NaN,NaN,2018-01-18,both,https://data.inpi.fr/entreprises/349599092,partiel
